# Detect Trucks Sentinel-2 - Europe
________________                                            

In [26]:
# load creds
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [27]:
bbox = [48.20, 16.30, 48.30, 16.46]

_______________
## 1 | Setup

In [28]:
# general
import os
from datetime import date, datetime, timedelta
import pandas as pd

# OSM API
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

# xcube
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube.core.maskset import MaskSet

# spatial
import numpy as np
import geopandas as gpd
from shapely import geometry, coords
from shapely.geometry import Polygon
#from osgeo import gdal #, gdal_array, ogr

# plotting
import matplotlib as plt
import IPython.display
%matplotlib inline

#### 1.1 Directories

In [29]:
dir_main = os.getcwd()
dir_not_commit = os.path.join(dir_main, "not_commit")
dir_ancil = os.path.join(dir_not_commit, "ancillary_data")
dirs = {"dir":dir_main, "dir_not_commit":dir_not_commit, "ancil":dir_ancil, "processing":os.path.join(dir_main, "processing"),
       "processed":os.path.join(dir_not_commit, "processed"), "ancil_roads":os.path.join(dir_ancil, "roads"), "ancil_gadm":os.path.join(dir_ancil, "gadm")}
for directory in list(dirs.values()):
    if not os.path.exists(directory): os.mkdir(directory)

#### 1.2 Files

In [30]:
files = {"gadm_efta":os.path.join(dirs["ancil_gadm"], "gadm36_0_EU_EFTA.gpkg"), 
         "gadm_europe":os.path.join(dirs["ancil_gadm"], "gadm36_0_europe.gpkg"), 
         "gadm_europe_union":os.path.join(dirs["ancil_gadm"], "gadm0_europe_union.gpkg"),
         "proc_grid":os.path.join(dirs["processing"], "processing_grid_gadm.geojson")}

____
## 2 | General Parameters

In [77]:
overwrite_results = True # shall already existing results be overwritten?

#### Cube

In [31]:
dataset = "S2L2A"
spatial_res = 0.00009 # 10m
bands = ["B02", "B03", "B04", "B08", "B11", "SCL"]
day_bins = "1D" # for cube
minimum_valid_observations = 15
grid_spacing = 1. # processing grid box size [degree]

#### Temporal

In [32]:
weekday = "wednesday" # process for this weekday
target = {"first":datetime(2020, 3, 16), "last":datetime(2020, 6, 6)}
baseline = {"first":datetime(2017, 12, 15), "last":datetime(2020, 3, target["first"].day - 1)}
n_days_sub = 91 # days per timestamp (sub-period)

#### OSM

In [33]:
osm_key = "highway"
osm_values = ["motorway", "trunk", "primary", "secondary", "tertiary"]
roads_buffer = 0.001 # degree

______
## 3 | Detection Parameters

In [34]:
thresholds = {"min_rgb":0.04,
              "max_red":0.15,
              "max_green":0.15,
              "max_blue":0.4,
              "max_ndvi":0.7,
              "max_ndwi":0.001,
              "max_ndsi":0.0001,
              "min_b11":0.05,
              "max_b11":0.55,
              "min_green_ratio":0.05,
              "min_red_ratio":0.1}

________________
## 4 | Utils

#### Names

In [57]:
def EPSG_4326(): return "EPSG:4326"
def EPSG_3857(): return "EPSG:3857"
def GEOJSON(): return "GeoJSON"
def GPKG(): return "GPKG"
def GEOJSON_EXT(): return ".geojson"
def GPKG_EXT(): return ".gpkg"
def NC_EXT(): return ".nc"
def BBOX_ID(): return("bbox_id")
def ABS_N_TRUCKS(): return("abs_n_trucks")
def SUM_TRUCKS(): return("sum_trucks")
def MEAN_N_TRUCKS(): return("mean_n_trucks")
def MEAN_N_TRUCKS_VEC(): return("mean_n_trucks_vec")
def N_OBS(): return("n_obs_fname")

#### File names

In [58]:
def fname_osm(bbox_id, osm_key, ext = GEOJSON_EXT()): return str(bbox_id) + "_" + osm_key + ext
def fname_abs_n_trucks(dirs_ts, bbox_id, ext = NC_EXT()): return os.path.join(dirs_ts[ABS_N_TRUCKS()], bbox_id + ext)
def fname_sum_trucks(dirs_ts, bbox_id, ext = NC_EXT()): return os.path.join(dirs_ts[SUM_TRUCKS()], bbox_id + ext)
def fname_mean_trucks(dirs_ts, bbox_id, ext = NC_EXT()): return os.path.join(dirs_ts[MEAN_N_TRUCKS()], bbox_id + ext)
def fname_mean_trucks_vec(dirs_ts, bbox_id, ext = GPKG_EXT()): return os.path.join(dirs_ts[MEAN_N_TRUCKS_VEC()], bbox_id + ext)
def fname_sum_obs(dirs_ts, bbox_id, ext = NC_EXT()): return os.path.join(dirs_ts[N_OBS], bbox_id + ext)

#### Directory structure

In [37]:
def make_dirs_ts(dir_ts):
    dir_ts_overall = os.path.join(dir_ts, "overall")
    dirs_ts = {ABS_N_TRUCKS():os.path.join(dir_ts, ABS_N_TRUCKS()), # acquisition-wise
               SUM_TRUCKS():os.path.join(dir_ts_overall, SUM_TRUCKS()) # aggregation
               MEAN_N_TRUCKS():os.path.join(dir_ts_overall, MEAN_N_TRUCKS()), # aggr
               MEAN_N_TRUCKS_VEC():os.path.join(dir_ts_overall, MEAN_N_TRUCKS_VEC()), # aggr
               N_OBS():os.path.join(dir_ts_overall, N_OBS())} # aggr
    for direc in dirs_ts.values():
        if not os.path.exists(direc): os.mkdir(direc)
    return dirs_ts

#### Processing grid
Create a grid covering Europe for processing chunk-wise.

In [38]:
def make_grid(grid_spacing, files):
    crs = EPSG_4326()
    gadm0_eu_efta = gpd.read_file(files["gadm_efta"])
    xmin,ymin,xmax,ymax = gadm0_eu_efta.total_bounds
    width = xmax - xmin
    height = ymax - ymin
    cols = int((width) / grid_spacing)
    rows = int((height) / grid_spacing)
    box_width = width / cols
    box_height = height / rows
    boxes = []
    for row in range(rows):
        for col in range(cols):
            col_right = col + 1
            row_lower = row + 1
            y_up = ymax-row*box_height
            y_low = ymax-row_lower*box_height
            x_left = xmin+col*box_width
            x_right = xmin+col_right*box_width
            boxes.append(Polygon([(x_left, y_up), 
                                  (x_right, y_up), 
                                  (x_right, y_low), 
                                  (x_left, y_low)]))
    grid = gpd.GeoDataFrame({"geometry":boxes})
    grid.crs = crs
    gadm0_europe = gpd.read_file(files["gadm_europe"])
    gadm0_europe.crs = crs
    file_union = files["gadm_europe_union"]
    if os.path.exists(file_union):
        gadm0_europe_clip_union = gpd.read_file(file_union)
    else:
        gadm0_europe["continent"] = ["EUROPE"] * len(gadm0_europe)
        gadm0_europe_clip = gpd.overlay(gadm0_europe, gpd.GeoDataFrame({"a":[1],"geometry":test}), how = "intersection")
        gadm0_europe_clip_union = gadm0_europe_clip.dissolve(by = "continent")
        gadm0_europe_clip_union.to_file(file_union, driver = GPKG())
    # intersect with gadm for cutting boxes at coast line
    grid_intersect = gpd.overlay(grid, gadm0_europe_clip_union, how = "intersection")
    # get country attributes
    grid_gadm = gpd.sjoin(grid_intersect, gadm0_europe, how = "left", op = "intersects")
    grid_gadm[BBOX_ID()] = range(len(grid_gadm))
    boxes = grid_gadm.geometry.apply(lambda geom : geom.bounds)
    grid_gadm.geometry = [Polygon(geometry.box(b[0], b[1], b[2], b[3])) for b in boxes] # use the light bboxes
    delete_cols = ["GID_0_left", "NAME_0_left", "a", "continent", "index_right"]
    for column in delete_cols:
        if column in grid_gadm.columns: 
            grid_gadm = grid_gadm.drop(column, 1)
    grid_gadm = grid_gadm.rename(columns = {"GID_0_right":"GID_0", "NAME_0_right":"NAME_0"})
    grid_gadm = grid_gadm[grid_gadm["GID_0"] != "RUS"] # do not include
    grid_gadm.to_file(files["proc_grid"], driver = GEOJSON())
    return grid_gadm

#### Vectors

#### Cubes

#### Arrays

In [53]:
# data np array
# lon_lat dict of:
### "lon": np array longitude values
### "lat": np array latitude values
def create_xy_xarray(data, lon_lat):
    return xr.DataArray(data, coords = lon_lat, dims = ("lat", "lon"))

# extracts coordinates at value in np array and returns points as GeoDataFrame
# data 2d np array
# match_value Float value in data where point coordinates are extracted
# lon_lat dict of:
### "lon": np array longitude values
### "lat": np array latitude values
def points_from_np(data, match_value, lon_lat):
    indices = np.argwhere(data == match_value)
    if len(indices) > 0:    
        lat_indices = indices[,[0]]
        lon_indices = indices[,[1]]
        lat_coords = lon_lat["lat"][lat_indices]
        lon_coords = lon_lat["lon"][lon_indices]
        points = gpd.GeoDataFrame(geometry = gpd.points_from_xy(lon_coords, lat_coords))
        return points

#### Dates

In [39]:
# date Datetime object to be checked
# weekday String weekday to be checked against
# returns Boolean if date is weekday
def is_weekday(date, weekday):
    weekday = weekday.lower()[0:3]
    y, m, d = 2000, 1, 3
    wd = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
    ref = {}
    for i in range(len(wd)): ref[wd[i]] = datetime(y, m, d + i)        
    return ((date - ref[weekday]) % 7) == 0 # check if date is in a sequence of 7

# Calculates the dates of the periods representing a timestamp each
# n_days_sub Integer how many dates shall one timestamp (sub-period) cover
# baseline Dict start and end dates overall baseline period
# target Dict start and end dates target period
def calc_periods(n_days_sub, baseline, target):
    fst, lst = "first", "last"
    base_first = baseline[fst]
    n_days = target[fst] - base_first
    n_sub = int(n_days.days / n_days_sub) # number of timestamps
    start = [base_first] * n_sub
    start = [start[i] + timedelta(n_days_sub * i) for i in range(len(start))]
    end = [start[i+1] - timedelta(1) for i in range(len(start)-1)]
    end.append(baseline[lst])
    periods = {"ts":range(len(start)), fst:start, lst:end}
    return periods

# formats a date for use in xcube
def datetime_to_np(datetime):
    return np.datetime64(datetime)

#### OSM data
Utils for retrieving road data from OSM through its API

In [40]:
# bbox List of four coords
# bbox_id Integer processing id of bbox
# osm_value String OSM value
# osm_key String OSM key
# element_type List of String
# returns GeoPandasDataFrame
def get_osm(bbox, 
            bbox_id,
            osm_value = "motorway",
            osm_key = "highway", # in OSM 'highway' contains several road types: https://wiki.openstreetmap.org/wiki/Key:highway
            element_type = ["way", "relation"]):
    
    quot = '"'
    select = quot+osm_key+quot + '=' + quot+osm_value+quot
    select_link = select.replace(osm_value, osm_value + "_link") # also get road links
    geoms = []
    for selector in [select, select_link]:  
        try:
            query = overpassQueryBuilder(bbox=bbox, 
                                         elementType=element_type, 
                                         selector=selector, 
                                         out='body',
                                         includeGeometry=True)
            elements = Overpass().query(query).elements()
            # create multiline of all elements
            if len(elements) > 0:
                for i in range(len(elements)):
                    elem = elements[i]
                    if elem.tags()[osm_key] == osm_value: geoms.append(elem.geometry())
        except:
            Warning("Could not retrieve " + select)
    try:
        lines = gpd.GeoDataFrame(crs = EPSG_4326(), geometry = geoms)
        n = len(geoms)
        lines[BBOX_ID()] = [bbox_id]*n
        lines["osm_value"] = [osm_value]*n # add road type
        return lines
    except:
        Warning("Could not merge " + osm_value)

In [52]:
# buffer Float road buffer distance [m]
# bbox List of four coords
# bbox_id Integer processing id of bbox
# osm_values List of String OSM values
# osm_key String OSM key
# buffer Float buffer width
# dir_write
def get_roads(bbox, bbox_id, osm_values, osm_key, buffer, dir_write):
    fwrite = os.path.join(dir_write, fname_osm(bbox_id, osm_key))
    roads = []
    for osm_value in osm_values:
        try:
            roads_osm = get_osm(bbox = bbox, bbox_id = bbox_id, osm_value = osm_value)
        except:
            Warning("'get_osm'" + "failed for bbox_id "+ str(bbox_id) + "osm_value " + osm_value + "osm_key" + osm_key)
        roads.append(roads_osm)
    roads_merge = gpd.GeoDataFrame(pd.concat(roads, ignore_index=True), crs=roads[0].crs)
    roads_buff = roads_merge.buffer(buffer)
    roads_buff.to_file(fwrite, driver = GEOJSON())
    return fwrite

_________
## 5 | Truck detection method

In [42]:
class TruckDetector():
    def __init__(self, band_stack):
        self.band_stack = band_stack
        self.B02 = band_stack.B02
        self.B03 = band_stack.B03
        self.B04 = band_stack.B04
        self.B08 = band_stack.B08
        self.B11 = band_stack.B11
        self.no_truck_mask = None # xarray
        self.trucks = None # xarray
    
    # Calculate a binary mask where pixels that are definitely no trucks are represented as 0.
    # thresholds Dict with at least:
    ### max_ndvi Float above this val: no trucks. For Vegetation
    ### max_ndwi Float above this val: no trucks. For Water
    ### max_ndsi Float above this val: no_trucks. For Snow
    ### min_rgb Float above this val: no_trucks. For dark surfaces, e.g. shadows
    ### max_blue Float above this val: no_trucks
    ### max_green Float above this val: no trucks
    ### max_red Float above this val: no trucks
    ### min_b11 Float below this val: no trucks. For dark surfaces, e.g. shadows
    ### max_b11 Float below this val: no trucks. For bright (sealed) surfaces, e.g. buildings
    def calc_no_trucks(self, thresholds):
        B02 = self.B02
        B03 = self.B03
        B04 = self.B04
        B08 = self.B08
        B11 = self.B11
        min_rgb = thresholds["min_rgb"]
        max_blue = thresholds["max_blue"]
        max_green = thresholds["max_green"]
        max_red = thresholds["max_red"]
        max_b11 = thresholds["max_b11"]
        ndvi_mask = ((B08 - B04) / (B08 + B04)) < thresholds["max_ndvi"]
        ndwi_mask = ((B02 - B11) / (B02 + B11)) < thresholds["max_ndwi"]
        ndsi_mask = ((B03 - B11) / (B03 + B11)) < thresholds["max_ndsi"]
        low_rgb_mask = (B02 > min_rgb) * (B03 > min_rgb) * (B04 > min_rgb)
        high_rgb_mask = (B02 < max_blue) * (B03 < max_green) * (B04 < max_red)
        b11_mask = ((B11 - B03) / (B11 + B03)) < max_b11
        b11_mask_abs = (B11 > thresholds["min_b11"]) * (B11 < max_b11)
        self.no_truck_mask = ndvi_mask * ndwi_mask * ndsi_mask * low_rgb_mask * high_rgb_mask * b11_mask * b11_mask_abs
    
    # Calculate a binary mask where trucks are represented as 1 and no trucks as 0.
    # thresholds Dict with at least:
    ### min_green_ratio Float, minimum value of blue-green ratio
    ### min_red_ratio Float, minimum value of blue-red ratio
    def detect_trucks(self, thresholds):
        B02 = self.B02
        B03 = self.B03
        B04 = self.B04
        bg_ratio = (B02 - B03) / (B02 + B03)
        br_ratio = (B02 - B04) / (B02 + B04)
        bg = (bg_ratio * self.no_trucks) > thresholds["min_green_ratio"]
        br = (br_ratio * self.no_trucks) > thresholds["min_red_ratio"]
        self.trucks = bg * br

_________
## 5 | Processing

In [43]:
class PeriodProcessor():
    def __init__(self, start, end, bbox, bbox_id):
        self.start = start
        self.end = end
        self.bbox = bbox
        self.bbox_id = bbox_id
        self.cube = None
        self.dates = None
        self.lon_lat = None
        self.n_observations = []
        self.detections = []
        self.mean_trucks = None
        self.sum_trucks = None
        self.sum_obs = None
        self.mean_trucks_points = None
    
    def get_cube(self, dataset, bands, tile_size, spatial_res, day_bins):
        config = CubeConfig(dataset_name = dataset,
                            band_names = bands,
                            tile_size = tile_size,
                            geometry = self.bbox,
                            time_range = [self.start, self.end],
                            spatial_res = spatial_res)
        cube = open_cube(config)
        self.cube = cube
        self.dates = cube.time.values
        self.lon_lat = {"lon":cube.cube.lon.values, "lat":cube.cube.lat.values}
        
    def mask_cube(self):
        scl = MaskSet(self.cube.SCL)
        high_prob = scl.clouds_high_probability
        med_prob = scl.clouds_medium_probability
        low_prob = scl.clouds_low_probability_or_unclassified
        cirrus = scl.cirrus
        cloud_mask = (high_prob + med_prob + low_prob + cirrus) == 0
        self.cube = cube.where(cloud_mask)
    
    # Add from acquisition methods
    def add_n_observations(self, band):
        obs = np.count_nonzero(np.isnan(band.values))
        obs.dtype = np.uint16
        self.n_observations.append(obs)
    
    def add_detections(self, trucks):
        self.detections.append(trucks)
    
    # Summarize methods
    def sum_trucks(self):
        self.sum_trucks = np.array(self.detections, dtype=np.unint16).sum(axis=0)
    
    def sum_obs(self):
        self.sum_obs = np.array(self.n_observations, dtype=np.uint16).sum(axis=0)
            
    def mean_trucks(self): 
        mean_trucks = np.round(self.sum_trucks / self.n_obs)
        self.mean_trucks = mean_trucks.astype(np.uint16)
                
    def vectorize_mean_trucks(self, dirs_ts, bbox_id):
        self.mean_trucks_points = points_from_np(self.mean_trucks, self.lon_lat)
    
    # Write methods
    def write_n_observations(self, dirs_ts, bbox_id, ext):
        fname = fname_sum_obs(dirs_ts, bbox_id, ext)
        sum_obs_xr = create_xy_xarray(self.sum_obs, self.lon_lat)
        sum_obs_xr.to_netcdf(sum_obs_xr, fname)
        
    def write_sum_trucks(self, dirs_ts, bbox_id, ext):
        fname = fname_sum_trucks(dirs_ts, bbox_id, ext)
        sum_xr = create_xy_xarray(self.sum_trucks, self.lon_lat)
        sum_xr.to_netcdf(sum_xr, fname)
    
    def write_mean_trucks(self, dirs_ts, bbox_id, ext):
        fname = fname_mean_trucks(dirs_ts, bbox_id, ext)
        mean_xr = create_xy_xarray(self.mean_trucks, self.lon_lat)
        mean_xr.to_netcdf(mean_xr, fname)
        
    def write_mean_trucks_vec(self, dirs_ts, bbox_id, ext):
        fname = fname_mean_trucks_vec(dirs_ts, bbox_id, ext)
        driver = GPKG() if ext == GPKG_EXT()
        driver = GEOJSON() if ext = GEOJSON_EXT()
        trucks_points.to_file(fname, driver = )
    
    def wrap_period(self, dirs_ts, bbox_id, ext_arr = NC_EXT(), ext_vec = GPKG_EXT()):
        period.sum_obs()
        period.sum_trucks()
        period.mean_trucks()
        try:
            period.vectorize_mean_trucks()
        except:
            Warning("points_from_np failed")
        period.write_n_observations(dirs_ts, bbox_id, ext_arr)
        period.write_sum_trucks(dirs_ts, bbox_id, ext_arr)
        period.write_mean_trucks(dirs_ts, bbox_id, ext_arr)
        period.write_mean_trucks_vec(dir_ts, bbox_id, ext_vec)

IndentationError: expected an indented block (<ipython-input-43-7a19a62a3c63>, line 46)

In [ ]:
class AcquisitionProcessor():
    def __init__(self, date_np64, cube):
        self.date_np64 = date_np64
        self.cube = cube
        self.band_stack = cube.sel(date_np64)
        self.detector = TruckDetector(self.band_stack)
    
    # percentage Float secifies the percentage of valid observations
    # that is needed to be considered as valid acquisition
    def has_observations(self, minimum_valid_percentage):
        B02 = self.band_stack.B02
        values = B02.values.flatten()
        n_vals = len(values)
        n_nan = np.count_nonzero(np.isnan(values)) # nonzero = isnan
        percent_valid = 100 - ((n_nan / n_vals) * 100)
        return percent_valid >= minimum_valid_percentage
    
    def do_detection(self, thresholds):     
        self.detector.calc_no_trucks(thresholds)
        self.detector.detect_trucks(thresholds)
        
    def write_detections(self, dirs_ts, bbox_id, ext = NC_EXT()):
        fname = fname_abs_n_trucks(dirs_ts, bbox_id, ext)
        trucks_np = self.detector.trucks.astype(np.uint16)
        trucks_xr = create_xy_xarray(trucks_np, {"lon":cube.lon.values, "lat":cube.lat.values})
        trucks_xr.to_netcdf(fname)

__________
____________
## Do Processing
Process data by __weekday__, __timestamp__ (sub period) processing grid __box__ (bbox_id).

#### Make or read processing grid

In [44]:
if os.path.exists(files["proc_grid"]):
    try:
        grid_gadm = gpd.read_file(files["proc_grid"])
    except:
        raise Exception("Failed reading proc grid from: " + files["proc_grid"])
else:
    grid_gadm = make_grid(grid_spacing, files)

#### Calc temporal bounds of baseline sub-periods

In [359]:
periods = calc_periods(n_days_sub, baseline, target)
periods["first"].append(target["first"]) # append start date of target period
periods["last"].append(target["last"]) # append end date of target period

#### Main

In [361]:
trace = []
ext_arr = NC_EXT()
ext_vec = GPKG_EXT()
for i in range(len(grid_gadm)):
    bbox = grid_gadm.geometry[i].bounds
    bbox_id = grid_gadm[BBOX_ID()][i]
    print("Processing: " + str(i))
    print("bbox_id: " + str(bbox_id))
    try:
        file_osm = get_roads(bbox, bbox_id, osm_values, osm_key, buffer, dirs["ancil_roads"])
    except:
        msg = "Could not get OSM roads: " + str(bbox_id)
        Warning(msg)
        trace.append(msg)
        continue
    first = periods["first"]
    last = periods["last"]
    for start, end in zip(first, last):
        ts = first.index(start)
        dir_ts = os.path.join(dirs["processed"], "ts_%s_%s_%s" %(str(ts), str(first), str(last)))
        if not os.path.exist(dir_ts) os.mkdir(dir_ts)
        dirs_ts = make_dirs_ts(dir_ts)
        print("TS: %s Start: %s End: %s" %(str(ts), str(start), str(end)))
        period = PeriodProcessor(start, end, bbox, bbox_id)
        period.get_cube(dataset, bands, tile_size, spatial_res, day_bins)
        period.mask_cube()
        n_acquisitions = 0
        for date in period.dates:
            date_np64 = datetime_to_np(date)
            acquisition = AcquisitionProcessor(date_np64, period.cube)
            if is_weekday(date, weekday) and acquisition.has_observations(minimum_valid_observations):
                n_acquisitions += 1
                acquisition.do_detection(thresholds)
                acquisition.write_detections(dirs_ts, bbox_id, ext_arr)                
                period.add_n_observations(band_stack.B02)
                period.add_detections(detector.trucks.values)
            else:
                continue
        if n_acquisitions > 0:
            period.wrap_period(dirs_ts, bbox_id, arr_ext, ext_vec)
        else:
            msg = "No acquisitions in period %s to %s. In bbox_id: %s" %(str(start), str(end), str(bbox_id))
            Warning(msg)
            trace.append(msg)
    print("Done with bbox_id: " + str(bbox_id))
    